# We want to do a run with no spatial covariance accounted for whatsoever

In [1]:
# !/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
sns.set_palette('colorblind')
matplotlib.rc('xtick', labelsize=15)
matplotlib.rc('ytick', labelsize=15)
matplotlib.rc('axes',labelsize=15)
matplotlib.rcParams['text.usetex'] = False

import pandas as pd
import pystan
import corner

import pickle
import glob

import os
import sys

__outdir__ = os.path.expanduser('~')+'/PhD/Gaia_Project/Output/Parallax_Runs/Highruns/'
__datdir__ = os.path.expanduser('~')+'/PhD/Gaia_Project/data/KepxDR2/'

In [2]:
nocov = '''
functions {
    real bailerjones_lpdf(real r, real L){
        return log((1/(2*L^3)) * (r*r) * exp(-r/L));
    }
}
data {
    int<lower = 0> N;
    vector[N] m;
    vector<lower=0>[N] m_err;
    vector[N] oo;
    vector<lower=0>[N] oo_err;
    vector<lower=0>[N] RlEbv;

    real oozp_init;
    real oozp_spread;
    real muH;
}
parameters {
    //Hyperparameters
    real mu;
    real<lower=0.> sigma;
    real<lower=1.> sigo;
    real<lower=0.5,upper=1.> Q;
    real<lower=.1, upper=4000.> L;
    real oo_zp;

    //Latent parameters
    vector[N] M_infd_std;
    vector[N] Ai;
    vector<lower = 1.>[N] r_infd;
}
transformed parameters{
    //Inferred and transformed parameters
    vector[N] M_infd;

    //Operations
    for (n in 1:N){
        M_infd[n] = mu + sigma * M_infd_std[n]; //Rescale the M fit
    }
}
model {
    //Define calculable properties
    vector[N] m_true;
    vector[N] oo_true;

    //Hyperparameters [p(theta_rc, L)]
    mu ~ normal(muH, 1.0); // Prior from seismo
    sigma ~ normal(0.0, 1.0);
    Q ~ normal(1., .25);
    sigo ~ normal(3.0, 1.0);
    L ~ uniform(0.1, 4000.);   // Prior on the length scale
    oo_zp ~ normal(oozp_init, oozp_spread); // Prior on the offset (in mu as)

    //Latent parameters [p(alpha_i | theta_rc, L)]
    Ai ~ normal(RlEbv, 0.05);
    for (n in 1:N){
        r_infd[n] ~ bailerjones(L);
        target += log_mix(Q,
            normal_lpdf(M_infd_std[n] | 0., 1.),
            normal_lpdf(M_infd_std[n] | 0., sigo));
    }

    //Calculable properties
    for (n in 1:N){
        m_true[n] = M_infd[n] + 5*log10(r_infd[n]) - 5 + Ai[n];
        oo_true[n] = (1000./r_infd[n]) + (oo_zp/1000.);
    }

    //Observables [p(D | theta_rc, L, alpha)]
    oo ~ normal(oo_true, oo_err);
    m ~ normal(m_true, m_err); //Measurement uncertainty on magnitude
}

'''
sm = pystan.StanModel(model_code = nocov, model_name='nocov')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL nocov_1388c4aa6535899f63f2c3ccb5867a25 NOW.


In [3]:
def read_data():
    sfile = __datdir__+'rcxyu18.csv'
    df = pd.read_csv(sfile)
    return df

In [4]:
df = read_data()

In [5]:
from omnitool.literature_values import hawkvals

band='GAIA'

rlebv = df.Ebv.values * 2.740 #As per Casagrande & Vandenberg 2018b

#Correct the Gaia G mags as per Casagrande & Vandenberg 2018b
mband = np.ones(len(df)) * df.GAIAmag.values
sel = (mband > 6.) & (mband < 16.5)
mband[sel] = 0.0505 + 0.9966*mband[sel]
merr = np.ones(len(mband)) * 10.e-3 #Setting precision to 10mmag by default

dat = {'N':len(df),
        'm': mband,
        'm_err': merr,
        'oo': df.parallax.values,
        'oo_err': df.parallax_error.values,
        'RlEbv': rlebv,
        'muH': hawkvals[band],
        'oozp_init' : 0.,
        'oozp_spread' : 1000.}

init= {'mu': hawkvals[band],
        'sigma': 0.1,
        'Q': 0.95,
        'sigo': 4.,
        'L': 1000.,
        'oozp':0.,
        'r_infd':df.r_est,
        'Ai':rlebv}

pars = ['mu', 'sigma', 'Q', 'sigo', 'L', 'oo_zp']
verbose = [r'$\mu_{RC} (mag)$',r'$\sigma_{RC} (mag)$',r'$Q$', r'$\sigma_o (mag)$', r'$L (pc)$', r'$\varpi_{zp} (\mu as)$']

In [ ]:
runlabel = __outdir__+band+'_nocov'

fit = sm.sampling(data = dat, iter = 5000, chains=4, seed = 24601,
                    init = [init for i in range(4)])

chain = np.array([fit[label] for label in pars])
np.savetxt(runlabel+'_chains.txt', chain)

pardict = {label:np.median(fit[label]) for label in pars}
pardict.update({label+'_std':np.std(fit[label]) for label in pars})
pardict = pd.DataFrame.from_dict(pardict, orient='index').T
pardict.to_csv(runlabel+'_pars.csv')

s = fit.summary()
rhat = s['summary'][:,-1]
np.savetxt(runlabel+'_rhats.txt',rhat)

corner.corner(chain.T, labels=verbose,quantiles=[0.16, 0.5, 0.84],show_titles=True, title_kwargs={"fontsize":12})
plt.savefig(runlabel+'_corner.png')
plt.close()

fit.plot()
plt.show()